![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20RoBertaForZeroShotClassification.ipynb)

## Import RoBertaForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 4.4.2` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import RoBerta models trained/fine-tuned for sequence classification via `RobertaForSequenceClassification` or `TFRobertaForSequenceClassification`. We can use these models for zero-shot classification.
  - These models are usually under `Sequence Classification` category and have `roberta` in their labels
  - For zero-shot classification, We will usually use models trained on the nli data sets for best performance.
- Reference: [TFRobertaForSequenceClassification](https://huggingface.co/docs/transformers/v4.29.1/en/model_doc/roberta#transformers.TFRobertaForSequenceClassification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 104.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tens

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [cross-encoder/nli-roberta-base](cross-encoder/nli-roberta-base) model from HuggingFace as an example
  - For zero-shot classification, We will usually use models trained on the (m)nli data set for best performance.
- In addition to `TFRobertaForSequenceClassification` we also need to save the `RobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer 
import tensorflow as tf

MODEL_NAME = 'cross-encoder/nli-roberta-base'

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFRobertaForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFRobertaForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")       
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {MODEL_NAME}

total 487204
-rw-r--r-- 1 root root       883 Jun  6 12:50 config.json
drwxr-xr-x 3 root root      4096 Jun  6 12:50 saved_model
-rw-r--r-- 1 root root 498881344 Jun  6 12:50 tf_model.h5


In [7]:
!ls -l {MODEL_NAME}/saved_model/1

total 9436
drwxr-xr-x 2 root root    4096 Jun  6 12:50 assets
-rw-r--r-- 1 root root      55 Jun  6 12:50 fingerprint.pb
-rw-r--r-- 1 root root  167553 Jun  6 12:50 keras_metadata.pb
-rw-r--r-- 1 root root 9481842 Jun  6 12:50 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun  6 12:50 variables


In [8]:
!ls -l {MODEL_NAME}_tokenizer

total 1432
-rw-r--r-- 1 root root 456318 Jun  6 12:48 merges.txt
-rw-r--r-- 1 root root    957 Jun  6 12:48 special_tokens_map.json
-rw-r--r-- 1 root root   1501 Jun  6 12:48 tokenizer_config.json
-rw-r--r-- 1 root root 999355 Jun  6 12:48 vocab.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing to convert `vocab.json` to `vocab.txt` from the tokenizer
- All we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [9]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in tokenizer.get_vocab().keys():
        f.write("%s\n" % item)

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}
!cp {MODEL_NAME}_tokenizer/merges.txt {asset_path}

In [10]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [11]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 852
-rw-r--r-- 1 root root     32 Jun  6 13:07 labels.txt
-rw-r--r-- 1 root root 456318 Jun  6 13:07 merges.txt
-rw-r--r-- 1 root root 407065 Jun  6 13:07 vocab.txt


## Import and Save RoBertaForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [15]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `RoBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [16]:
from sparknlp.annotator import *
from sparknlp.base import *

zero_shot_classifier = RoBertaForZeroShotClassification.loadSavedModel(
    '{}/saved_model/1'.format(MODEL_NAME),
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "not urgent", "phone", "tablet", "computer"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [17]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your RoBertaForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [18]:
! ls -l {MODEL_NAME}_spark_nlp

total 496104
drwxr-xr-x 6 root root      4096 Jun  6 13:12 fields
drwxr-xr-x 2 root root      4096 Jun  6 13:12 metadata
-rw-r--r-- 1 root root 507995708 Jun  6 13:12 roberta_classification_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForSequenceClassification model 😊 

In [23]:
zero_shot_classifier_loaded = RoBertaForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCandidateLabels(["urgent", "not urgent", "phone", "tablet", "computer"])

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [24]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to harry up and get to the airport. We are going to miss our flight!"]]

# create a DataFrame in PySpark
inputDataset = spark.createDataFrame(text, ["text"])
model = pipeline.fit(inputDataset)
model.transform(inputDataset).select("class.result").show()

+------------+
|      result|
+------------+
|  [computer]|
|     [phone]|
|  [computer]|
|     [phone]|
|     [phone]|
|[not urgent]|
|     [phone]|
+------------+



That's it! You can now go wild and use hundreds of `RoBertaForSequenceClassification` models as zero-shot classifiers from HuggingFace 🤗 in Spark NLP 🚀 